In [419]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [69]:
data = pd.read_csv('/Users/viktorialavrinenko/Desktop/Data Science/Доки/movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
105,tt3862750,12000000,60273173,The Perfect Guy,Michael Ealy|Sanaa Lathan|Morris Chestnut|Kath...,David M. Rosenthal,"Trust one, fear the other.",Leah Vaughn appears to have the ideal life. Sh...,100,Drama|Thriller,Screen Gems,9/11/2015,5.6,2015
250,tt0409459,130000000,185258983,Watchmen,Malin Ã…kerman|Billy Crudup|Carla Gugino|Jeffr...,Zack Snyder,Justice is coming to all of us. No matter wha...,In a gritty and alternate 1985 the glory days ...,162,Action|Mystery|Science Fiction,Paramount Pictures|DC Comics|Lawrence Gordon P...,3/5/2009,6.8,2009
1030,tt1212450,26000000,53676580,Lawless,Shia LaBeouf|Tom Hardy|Guy Pearce|Jessica Chas...,John Hillcoat,"When the law became corrupt, outlaws became he...","Set in the Depression-era Franklin County, Vir...",115,Crime|Drama,The Weinstein Company|Benaroya Pictures|Yucaip...,8/29/2012,6.7,2012
100,tt2967224,35000000,51680201,Hot Pursuit,Reese Witherspoon|SofÃ­a Vergara|Michael Mosle...,Anne Fletcher,Armed and Sort-of Dangerous.,An uptight by-the-book cop must protect the wi...,87,Action|Crime|Comedy,New Line Cinema|Warner Bros.|Metro-Goldwyn-May...,5/8/2015,5.5,2015
1632,tt0372588,32000000,50907422,Team America: World Police,Trey Parker|Matt Stone|Kristen Miller|Masasa M...,Trey Parker,Putting The 'F' back in 'Freedom',Team America World Police follows an internati...,98,Music|Adventure|Animation|Action|Comedy,Paramount Pictures|Scott Rudin Productions,10/11/2004,6.5,2004


In [464]:
data.describe()

,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


# Предобработка

In [6]:
answers = {} # создадим словарь для ответов

In [459]:
# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

data['profit'] = data['revenue'] - data['budget']
#data['genres_split'] = list(map(lambda x: x.split(sep = '|'), data['genres']))
data['genres_split'] = data.genres.str.split('|')
data['director_split'] = data.director.str.split('|')
data['cast_split'] = data.cast.str.split('|')
data['production_companies_split'] = data.production_companies.str.split('|')
data['release_date_split'] = data.release_date.str.split('/')
data['release_month'] = list(map(lambda x: x[0], data.release_date_split))

In [460]:
data.head(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_split,director_split,cast_split,production_companies_split,release_date_split,release_month
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,...,6/9/2015,6.5,2015,1363528810,"[Action, Adventure, Science Fiction, Thriller]",[Colin Trevorrow],"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...","[Universal Studios, Amblin Entertainment, Lege...","[6, 9, 2015]",6
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,...,5/13/2015,7.1,2015,228436354,"[Action, Adventure, Science Fiction, Thriller]",[George Miller],"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...","[Village Roadshow Pictures, Kennedy Miller Pro...","[5, 13, 2015]",5
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,...,3/18/2015,6.3,2015,185238201,"[Adventure, Science Fiction, Thriller]",[Robert Schwentke],"[Shailene Woodley, Theo James, Kate Winslet, A...","[Summit Entertainment, Mandeville Films, Red W...","[3, 18, 2015]",3
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,...,12/15/2015,7.5,2015,1868178225,"[Action, Adventure, Science Fiction, Fantasy]",[J.J. Abrams],"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...","[Lucasfilm, Truenorth Productions, Bad Robot]","[12, 15, 2015]",12
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,...,4/1/2015,7.3,2015,1316249360,"[Action, Crime, Thriller]",[James Wan],"[Vin Diesel, Paul Walker, Jason Statham, Miche...","[Universal Pictures, Original Film, Media Righ...","[4, 1, 2015]",4


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [11]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
#answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [465]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['imdb_id','original_title','budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


ВАРИАНТ 2

In [468]:
# можно добавлять разные варианты решения
data.groupby(['budget'])[['imdb_id','original_title']].max().sort_index(ascending=False).head(1)

,imdb_id,original_title
budget,,
380000000,tt1298650,Pirates of the Caribbean: On Stranger Tides


# 2. Какой из фильмов самый длительный (в минутах)?

In [14]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)'

In [469]:
data[data.runtime == data.runtime.max()][['imdb_id','original_title','runtime']]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214


ВАРИАНТ 2

In [488]:
data.groupby(['runtime'])[['imdb_id','original_title']].max().sort_index(ascending=False).head(1)

,imdb_id,original_title
runtime,,
214,tt0279111,Gods and Generals


ВАРИАНТ 3

In [494]:
data.sort_values('runtime', ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_split,director_split,cast_split,production_companies_split,release_date_split,release_month
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,...,2/21/2003,5.8,2003,-43076064,"[Drama, History, War]",[Ronald F. Maxwell],"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke...","[Turner Pictures, Antietam Filmworks]","[2, 21, 2003]",2


# 3. Какой из фильмов самый короткий (в минутах)?





In [471]:
answers['3'] = 'Winnie the Pooh (tt1449283)'
data[data.runtime == data.runtime.min()][['imdb_id','original_title','runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


# 4. Какова средняя длительность фильмов?


In [472]:
answers['4'] = '110'
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [479]:
answers['5'] = '107'
#data.runtime.describe()
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [480]:
# лучше код получения столбца profit вынести в Предобработку что в начале
answers['6'] = 'Avatar (tt0499549)'
data[data.profit == data.profit.max()][['imdb_id','original_title','profit']]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


# 7. Какой фильм самый убыточный? 

In [481]:
answers['7'] = 'The Lone Ranger (tt1210819)'
data[data.profit == data.profit.min()][['imdb_id','original_title','profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [484]:
answers['8'] = '1478'
data[data.revenue > data.budget]['imdb_id'].count()

1478

ВАРИАНТ 2

In [486]:
data[data.profit > 0]['imdb_id'].count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [492]:
answers['9'] = 'The Dark Knight (tt0468569)'
data[(data.release_year == 2008)].groupby(['revenue']).max().sort_index(ascending=False).head(1)

,imdb_id,budget,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_split,director_split,cast_split,production_companies_split,release_date_split,release_month
revenue,,,,,,,,,,,,,,,,,,,,
1001921825,tt0468569,185000000,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825,"[Drama, Action, Crime, Thriller]",[Christopher Nolan],"[Christian Bale, Michael Caine, Heath Ledger, ...","[DC Comics, Legendary Pictures, Warner Bros., ...","[7, 16, 2008]",7


ВАРИАНТ 2

In [493]:
data[(data.release_year == 2008)].groupby(['revenue'])[['imdb_id','original_title']].max().sort_index(ascending=False).head(1)

,imdb_id,original_title
revenue,,
1001921825,tt0468569,The Dark Knight


ВАРИАНТ 3

In [491]:
data[data.release_year == 2008].sort_values('revenue', ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_split,director_split,cast_split,production_companies_split,release_date_split,release_month
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,7/16/2008,8.1,2008,816921825,"[Drama, Action, Crime, Thriller]",[Christopher Nolan],"[Christian Bale, Michael Caine, Heath Ledger, ...","[DC Comics, Legendary Pictures, Warner Bros., ...","[7, 16, 2008]",7


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [497]:
answers['10'] = 'The Lone Ranger (tt1210819)'
data[((data.release_year>=2012) & (data.release_year<=2014))].groupby(['profit']).min().head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,genres_split,director_split,cast_split,production_companies_split,release_date_split,release_month
profit,,,,,,,,,,,,,,,,,,,,
-165710090,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,"[Action, Adventure, Western]",[Gore Verbinski],"[Johnny Depp, Armie Hammer, William Fichtner, ...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[7, 3, 2013]",7


ВАРИАНТ 2

In [498]:
data[((data.release_year>=2012) & (data.release_year<=2014))].groupby(['profit'])[['imdb_id','original_title']].min().head(1)

,imdb_id,original_title
profit,,
-165710090,tt1210819,The Lone Ranger


# 11. Какого жанра фильмов больше всего?

In [499]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
answers['11'] = 'Drama'
Counter(data.genres_split.sum()).most_common(1)[0]

('Drama', 782)

ВАРИАНТ 2

In [500]:
data_genres = data.explode('genres_split')
data_genres['genres_split'].value_counts().head(1)

Drama    782
Name: genres_split, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [501]:
answers['12'] = 'Drama'
Counter(data[data.profit > 0].genres_split.sum()).most_common(1)[0]

('Drama', 560)

In [503]:
data_genres[data_genres.profit > 0]['genres_split'].value_counts().head(1)

Drama    560
Name: genres_split, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [504]:
answers['13'] = 'Peter Jackson'
data_director = data.explode('director_split')
data_director.groupby(['director_split']).revenue.sum().sort_values(ascending = False).head(1)

director_split
Peter Jackson    6490593685
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [506]:
answers['14'] = 'Robert Rodriguez'
data_genres = data.explode('genres_split')
data_genres_and_director = data_genres.explode('director_split')
#data_genres_and_director[data_genres_and_director.genres_split == 'Action'].groupby(['director_split'])['original_title'].count().sort_values(ascending = False).head(1)
data_genres_and_director[data_genres_and_director.genres_split == 'Action'].groupby(['director_split'])['imdb_id'].count().sort_values(ascending = False).head(1)

director_split
Robert Rodriguez    9
Name: imdb_id, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [507]:
answers['15'] = 'Chris Hemsworth'
data_cast = data.explode('cast_split')
data_cast[data_cast.release_year == 2012].groupby(['cast_split']).revenue.sum().sort_values(ascending = False).head(1)

cast_split
Chris Hemsworth    2027450773
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов? Примечание: в фильмах, где бюджет выше среднего по данной выборке.

In [508]:
answers['16'] = 'Matt Damon'
#data_cast[data_cast.budget > data_cast.budget.mean()].groupby(['cast_split'])['original_title'].count().sort_values(ascending = False).head(1)
data_cast[data_cast.budget > data_cast.budget.mean()].groupby(['cast_split'])['imdb_id'].count().sort_values(ascending = False).head(1)

cast_split
Matt Damon    18
Name: imdb_id, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [509]:
answers['17'] = 'Action'
data_cast_and_ganres = data_cast.explode('genres_split')
#data_cast_and_ganres[data_cast_and_ganres.cast_split == 'Nicolas Cage'].groupby(['genres_split'])['original_title'].count().sort_values(ascending = False).head(1)
data_cast_and_ganres[data_cast_and_ganres.cast_split == 'Nicolas Cage'].groupby(['genres_split'])['imdb_id'].count().sort_values(ascending = False).head(1)

genres_split
Action    17
Name: imdb_id, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [519]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'
data_production_companies = data.explode('production_companies_split')
data_production_companies[data_production_companies.production_companies_split == 'Paramount Pictures'].groupby(['imdb_id'])['profit'].min().sort_values().head(1)

imdb_id
tt0267626   -64831034
Name: profit, dtype: int64

ВАРИАНТ 2

In [520]:
data_production_companies[data_production_companies.production_companies_split == 'Paramount Pictures'][['imdb_id','original_title','profit']].sort_values('profit',ascending = True).head(1)

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034


ВАРИАНТ 3

In [521]:
data_production_companies[data_production_companies.production_companies_split == 'Paramount Pictures'].sort_values('profit',ascending = True)[['imdb_id','original_title']].head(1)

,imdb_id,original_title
925,tt0267626,K-19: The Widowmaker


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [522]:
answers['19'] = '2015'
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending = False).head(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [523]:
answers['20'] = '2014'
data_production_companies[data_production_companies.production_companies_split.str.contains('Warner Bros')].groupby(['release_year'])['profit'].sum().sort_values(ascending = False).head(1)

release_year
2014    2292949646
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [524]:
answers['21'] = 'Сентябрь'
#data.groupby(['release_month'])['original_title'].count().sort_values(ascending = False).head(1)
data.groupby(['release_month'])['imdb_id'].count().sort_values(ascending = False).head(1)

release_month
9    227
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [525]:
answers['22'] = '450'
#data[(data.release_month == '6') | (data.release_month == '7') | (data.release_month == '8')]['original_title'].count()
data[(data.release_month == '6') | (data.release_month == '7') | (data.release_month == '8')]['imdb_id'].count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [526]:
answers['23'] = 'Peter Jackson'
#data_director[(data_director.release_month == '12') | (data_director.release_month == '1') | (data_director.release_month == '2')].groupby(['director_split'])['original_title'].count().sort_values(ascending = False).head(1)
data_director[(data_director.release_month == '12') | (data_director.release_month == '1') | (data_director.release_month == '2')].groupby(['director_split'])['imdb_id'].count().sort_values(ascending = False).head(1)

director_split
Peter Jackson    7
Name: imdb_id, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [527]:
answers['24'] = 'Four By Two Productions'
data_production_companies['title_len'] = data_production_companies.original_title.apply(lambda x: len(x))
data_production_companies.groupby(['production_companies_split'])['title_len'].mean().sort_values(ascending = False).head(1)

production_companies_split
Four By Two Productions    83.0
Name: title_len, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [528]:
answers['25'] = 'Midnight Picture Show'
data_production_companies['overview_words_num'] = list(map(lambda x: len(x.split(' ')), data_production_companies['overview']))
data_production_companies.groupby(['production_companies_split'])['overview_words_num'].mean().sort_values(ascending = False).head(1)

production_companies_split
Midnight Picture Show    175.0
Name: overview_words_num, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [529]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
one_percent = data.quantile(0.99,numeric_only = True)['vote_average']
data[data.vote_average > one_percent][['original_title','vote_average']]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [530]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
print('Johnny Depp & Helena Bonham Carter:', data[(data.cast.str.contains('Johnny Depp'))&(data.cast.str.contains('Helena Bonham Carter'))]['imdb_id'].count())
print('Ben Stiller & Owen Wilson:', data[(data.cast.str.contains('Ben Stiller'))&(data.cast.str.contains('Owen Wilson'))]['imdb_id'].count())
print('Vin Diesel & Paul Walker:', data[(data.cast.str.contains('Vin Diesel'))&(data.cast.str.contains('Paul Walker'))]['imdb_id'].count())
print('Adam Sandler & Kevin James:', data[(data.cast.str.contains('Adam Sandler'))&(data.cast.str.contains('Kevin James'))]['imdb_id'].count())
print('Daniel Radcliffe & Rupert Grint:', data[(data.cast.str.contains('Daniel Radcliffe'))&(data.cast.str.contains('Rupert Grint'))]['imdb_id'].count())

Johnny Depp & Helena Bonham Carter: 6
Ben Stiller & Owen Wilson: 6
Vin Diesel & Paul Walker: 5
Adam Sandler & Kevin James: 5
Daniel Radcliffe & Rupert Grint: 8


ВАРИАНТ 2

In [543]:
pairs = Counter()
for i in data.cast_split:
    for j in list(combinations(i, 2)):
        if j not in pairs:
            pairs[j] = 1
        else:
            pairs[j] += 1

pairs.most_common(5)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6)]

# Submission

In [544]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [545]:
# и убедиться что ни чего не пропустил)
len(answers)

27